In [24]:
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
import sys
import numpy as np
# from sklearn import cross_validation
from sklearn.naive_bayes import GaussianNB
# from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neural_network import MLPClassifier
from sklearn import svm
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.neural_network import MLPClassifier
from tflearn.layers.normalization import local_response_normalization
# from tensorflow.contrib import learn
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn import preprocessing
import pandas as pd
max_features=10000
max_document_length=100
min_opcode_count=2


#pro
#webshell_dir="../Data/webshell/b/")
#whitefile_dir="../Data/webshell/w/")

whitefile_dir="../Data/webshell/normal/php/"
#webshell_dir="../Data/webshell/dev-b/")
#whitefile_dir="../Data/webshell/dev-w/")
check_dir="../Downloads/php-exploit-scripts-master/"
white_count=0
black_count=0
php_bin="/Users/liu.yan/Desktop/code/2book/opt/php/bin/php"
#php_bin="/Users/maidou/Desktop/book/2book/2book/opt/php/bin/php"
#php_bin=" /home/fuxi/dev/opt/php/bin/php"


pkl_file="webshell-opcode-cnn.pkl"

data_pkl_file="data-webshell-opcode-tf.pkl"
label_pkl_file="label-webshell-opcode-tf.pkl"


#pro
#php_bin="/home/fuxi/dev/opt/php/bin/php"


# 11.2　特征提取

## 11.2.1　词袋和TF-IDF模型

In [3]:
def load_file(file_path):
    t=""
    with open(file_path,errors='replace') as f:
        for line in f:
            line=line.strip('\n')
            t+=line
    return t

def load_files_re(dir):
    files_list = []
    g = os.walk(dir)
    for path, d, filelist in g:
        #print d;
        for filename in filelist:
            #print os.path.join(path, filename)
            if filename.endswith('.php') or filename.endswith('.txt'):
                fulepath = os.path.join(path, filename)
                print("Load %s" % fulepath)
                t = load_file(fulepath)
                files_list.append(t)

    return files_list

webshell_dir="../Data/webshell/webshell/PHP/"
WebShell_files_list=load_files_re(webshell_dir)
y1=[1]*len(WebShell_files_list)
black_count=len(WebShell_files_list)

Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\exp4php\hadsky.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\address.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\Da.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\dirs.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\dirs_prettyfied.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\farbtastic_cache.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\farbtastic_cache_unobfuscated_version1.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\indoxploit.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\plugin38.php
Load ../Data/webshell/webshel

In [4]:
wp_files_list=load_files_re(whitefile_dir)
y2=[0]*len(wp_files_list)
white_count=len(wp_files_list)

Load ../Data/webshell/normal/php/304.php
Load ../Data/webshell/normal/php/305.php
Load ../Data/webshell/normal/php/306.php
Load ../Data/webshell/normal/php/400.php
Load ../Data/webshell/normal/php/401.php
Load ../Data/webshell/normal/php/402.php
Load ../Data/webshell/normal/php/403.php
Load ../Data/webshell/normal/php/404.php
Load ../Data/webshell/normal/php/405.php
Load ../Data/webshell/normal/php/406.php
Load ../Data/webshell/normal/php/407.php
Load ../Data/webshell/normal/php/408.php
Load ../Data/webshell/normal/php/409.php
Load ../Data/webshell/normal/php/410.php
Load ../Data/webshell/normal/php/411.php
Load ../Data/webshell/normal/php/412.php
Load ../Data/webshell/normal/php/413.php
Load ../Data/webshell/normal/php/414.php
Load ../Data/webshell/normal/php/415.php
Load ../Data/webshell/normal/php/416.php
Load ../Data/webshell/normal/php/417.php
Load ../Data/webshell/normal/php/418.php
Load ../Data/webshell/normal/php/428.php
Load ../Data/webshell/normal/php/429.php
Load ../Data/web

In [13]:
pd.DataFrame({'white':wp_files_list,
              "white_y":y2})

,white,white_y
0,<?php/** * Exception for 304 Not Modified resp...,0
1,<?php/** * Exception for 305 Use Proxy respons...,0
2,<?php/** * Exception for 306 Switch Proxy resp...,0
3,<?php/** * Exception for 400 Bad Request respo...,0
4,<?php/** * Exception for 401 Unauthorized resp...,0
...,...,...
566,<?php/** * Disable error reporting * * Set thi...,0
567,<?php/** * Handle Trackbacks and Pingbacks Sen...,0
568,"<?phpglobal $wpcom_api_key, $akismet_api_host,...",0
569,<?php/** * Class used internally by Diff to ac...,0


In [14]:
pd.DataFrame({"black":WebShell_files_list,
              "black_y":y1})

,black,black_y
0,<?php/**Author:l34rnerDesc:HadSky CMS <=2.3.7 ...,1
1,"<?php ${""\x47\x4c\x4fB\x41\x4c\x53""}[""\x75\x65...",1
2,<?ignore_user_abort();set_time_limit(0);functi...,1
3,"<?php $r76=""F[<PAlDf|]}M@~79/O8Kx\rH6r&-c5k\n3...",1
4,"<?php$r76 = ""F[<PAlDf|]}M@~79/O8Kx\rH6r&-c5k\n...",1
...,...,...
552,"<?php$auth_pass = ""63a9f0ea7bb98050796b649e854...",1
553,"<?php$auth_pass = ""63a9f0ea7bb98050796b649e854...",1
554,<?php /*This is a private WSO Shell modificati...,1
555,<?php/* WSO 4.0.5 (Web Shell by HARD _LINUX) *...,1


### tf

In [ ]:
def  get_features_by_tf():
    global  max_document_length
    global white_count
    global black_count
    x=[]
    y=[]

    webshell_files_list = load_files_re(webshell_dir)
    y1=[1]*len(webshell_files_list)
    black_count=len(webshell_files_list)

    wp_files_list =load_files_re(whitefile_dir)
    y2=[0]*len(wp_files_list)

    white_count=len(wp_files_list)


    x=webshell_files_list+wp_files_list
    y=y1+y2

    vp=tflearn.data_utils.VocabularyProcessor(max_document_length=max_document_length,
                                              min_frequency=0,
                                              vocabulary=None,
                                              tokenizer_fn=None)
    x=vp.fit_transform(x, unused_y=None)
    x=np.array(list(x))
    return x,y


### tfidf

In [15]:
def get_feature_by_bag_tfidf():
    global white_count
    global black_count
    global max_features
    print("max_features=%d" % max_features)
    x=[]
    y=[]

    webshell_files_list = load_files_re(webshell_dir)
    y1=[1]*len(webshell_files_list)
    black_count=len(webshell_files_list)

    wp_files_list =load_files_re(whitefile_dir)
    y2=[0]*len(wp_files_list)

    white_count=len(wp_files_list)


    x=webshell_files_list+wp_files_list
    y=y1+y2
    # 使用2-Gram提取词袋模型，并使用TF-IDF进行处理
    CV = CountVectorizer(ngram_range=(2, 4), decode_error="ignore",max_features=max_features,
                                       token_pattern = r'\b\w+\b',min_df=1, max_df=1.0)
    x=CV.fit_transform(x).toarray()

    transformer = TfidfTransformer(smooth_idf=False)
    x_tfidf = transformer.fit_transform(x)
    x = x_tfidf.toarray()

    return x,y

x,y = get_feature_by_bag_tfidf()

max_features=10000
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\exp4php\hadsky.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\address.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\Da.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\dirs.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\dirs_prettyfied.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\farbtastic_cache.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\farbtastic_cache_unobfuscated_version1.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\indoxploit.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\plugin38.php
Load ../Da

In [17]:
pd.DataFrame(x)

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 11.2.2　opcode和N-Gram模型

In [19]:
import subprocess

def load_file_opcode(file_path):
    global php_bin
    t = ""
    cmd = f"{php_bin} -dvld.active=1 -dvld.execute=0 {file_path}"
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    output = result.stdout

    t = output
    tokens = re.findall(r'\s(\b[A-Z_]+\b)\s', output)
    t = " ".join(tokens)

    print(f"opcode count {len(tokens)}")
    return t

def load_files_opcode_re(dir):
    global min_opcode_count
    files_list = []
    g = os.walk(dir)
    for path, d, filelist in g:
        #print d;
        for filename in filelist:
            #print os.path.join(path, filename)
            if filename.endswith('.php') :
                fulepath = os.path.join(path, filename)
                print("Load %s opcode" % fulepath)
                t = load_file_opcode(fulepath)
                if len(t) > min_opcode_count:
                    files_list.append(t)
                else:
                    print("Load %s opcode failed" % fulepath)
                #print "Add opcode %s" % t

    return files_list

load_files_opcode_re(webshell_dir)

ModuleNotFoundError: No module named 'commands'

### opcode

In [ ]:
def get_feature_by_opcode():
    global white_count
    global black_count
    global max_features
    global webshell_dir
    global whitefile_dir
    print("max_features=%d webshell_dir=%s whitefile_dir=%s" % (max_features,webshell_dir,whitefile_dir))
    x=[]
    y=[]

    webshell_files_list = load_files_opcode_re(webshell_dir)
    y1=[1]*len(webshell_files_list)
    black_count=len(webshell_files_list)

    wp_files_list =load_files_opcode_re(whitefile_dir)
    y2=[0]*len(wp_files_list)

    white_count=len(wp_files_list)


    x=webshell_files_list+wp_files_list
    #print x
    y=y1+y2

    CV = CountVectorizer(ngram_range=(2, 4), decode_error="ignore",max_features=max_features,
                                       token_pattern = r'\b\w+\b',min_df=1, max_df=1.0)

    x=CV.fit_transform(x).toarray()

    return x,y


### opcode-tf

In [ ]:
def get_feature_by_opcode_tf():
    global white_count
    global black_count
    global max_document_length
    x=[]
    y=[]

    if os.path.exists(data_pkl_file) and os.path.exists(label_pkl_file):
        f = open(data_pkl_file, 'rb')
        x = pickle.load(f)
        f.close()
        f = open(label_pkl_file, 'rb')
        y = pickle.load(f)
        f.close()
    else:
        webshell_files_list = load_files_opcode_re(webshell_dir)
        y1=[1]*len(webshell_files_list)
        black_count=len(webshell_files_list)

        wp_files_list =load_files_opcode_re(whitefile_dir)
        y2=[0]*len(wp_files_list)

        white_count=len(wp_files_list)


        x=webshell_files_list+wp_files_list
        #print x
        y=y1+y2

        vp=tflearn.data_utils.VocabularyProcessor(max_document_length=max_document_length,
                                                  min_frequency=0,
                                                  vocabulary=None,
                                                  tokenizer_fn=None)
        x=vp.fit_transform(x, unused_y=None)
        x=np.array(list(x))

        f = open(data_pkl_file, 'wb')
        pickle.dump(x, f)
        f.close()
        f = open(label_pkl_file, 'wb')
        pickle.dump(y, f)
        f.close()
    #print x
    #print y
    return x,y

## 11.3　模型训练与验证

### 11.3.1　朴素贝叶斯算法

In [21]:
def do_metrics(y_test,y_pred):
    print("metrics.accuracy_score:")
    print(metrics.accuracy_score(y_test, y_pred))
    print("metrics.confusion_matrix:")
    print(metrics.confusion_matrix(y_test, y_pred))
    print("metrics.precision_score:")
    print(metrics.precision_score(y_test, y_pred))
    print("metrics.recall_score:")
    print(metrics.recall_score(y_test, y_pred))
    print("metrics.f1_score:")
    print(metrics.f1_score(y_test,y_pred))

def do_nb(x,y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)
    gnb = GaussianNB()
    gnb.fit(x_train, y_train)
    y_pred = gnb.predict(x_test)
    do_metrics(y_test,y_pred)

In [29]:
# x,y = get_feature_by_opcode_tf()
# x,y = get_feature_by_opcode()
x,y = get_feature_by_bag_tfidf()


max_features=10000
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\exp4php\hadsky.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\address.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\Da.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\dirs.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\dirs_prettyfied.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\farbtastic_cache.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\farbtastic_cache_unobfuscated_version1.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\indoxploit.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\plugin38.php
Load ../Da

In [30]:
do_nb(x,y)

metrics.accuracy_score:
0.9269911504424779
metrics.confusion_matrix:
[[231   7]
 [ 26 188]]
metrics.precision_score:
0.9641025641025641
metrics.recall_score:
0.8785046728971962
metrics.f1_score:
0.9193154034229829


### 11.3.2　深度学习算法之MLP

In [31]:
def do_mlp(x,y):
    #mlp
    clf = MLPClassifier(solver='lbfgs',
                        alpha=1e-5,
                        hidden_layer_sizes=(5, 2),
                        random_state=1)

    #print clf
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(y_train)
    print(y_pred)
    print(y_test)
    do_metrics(y_test,y_pred)


In [32]:
# x,y = get_feature_by_opcode_tf()
# x,y = get_feature_by_opcode()
# x,y = get_feature_by_bag_tfidf()

do_nb(x,y)

max_features=10000
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\exp4php\hadsky.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\address.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\Da.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\dirs.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\dirs_prettyfied.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\farbtastic_cache.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\farbtastic_cache_unobfuscated_version1.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\indoxploit.php
Load ../Data/webshell/webshell/PHP/mattiasgeniar\php-exploit-scripts-master\found_on_drupal\plugin38.php
Load ../Da

### 11.3.3　深度学习算法之CNN

In [37]:
def do_cnn(x,y):
    global max_document_length
    print("CNN and tf")
    trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.4, random_state=0)
    y_test=testY

    trainX = pad_sequences(trainX, maxlen=max_document_length, value=0.)
    testX = pad_sequences(testX, maxlen=max_document_length, value=0.)
    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=2)
    testY = to_categorical(testY, nb_classes=2)

    # Building convolutional network
    network = input_data(shape=[None,max_document_length], name='input')
    network = tflearn.embedding(network, input_dim=1000000, output_dim=128)
    branch1 = conv_1d(network, 128, 3, padding='valid', activation='relu', regularizer="L2")
    branch2 = conv_1d(network, 128, 4, padding='valid', activation='relu', regularizer="L2")
    branch3 = conv_1d(network, 128, 5, padding='valid', activation='relu', regularizer="L2")
    network = merge([branch1, branch2, branch3], mode='concat', axis=1)
    network = tf.expand_dims(network, 2)
    network = global_max_pool(network)
    network = dropout(network, 0.8)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=0.001,
                         loss='categorical_crossentropy', name='target')

    model = tflearn.DNN(network, tensorboard_verbose=0)
    #if not os.path.exists(pkl_file):
        # Training
    model.fit(trainX, trainY,
                  n_epoch=5, shuffle=True, validation_set=0.1,
                  show_metric=True, batch_size=100,run_id="webshell")
    #    model.save(pkl_file)
    #else:
    #    model.load(pkl_file)

    y_predict_list=model.predict(testX)
    #y_predict = list(model.predict(testX,as_iterable=True))

    y_predict=[]
    for i in y_predict_list:
        print( i[0])
        if i[0] > 0.5:
            y_predict.append(0)
        else:
            y_predict.append(1)
    print('y_predict_list:')
    print(y_predict_list)
    print('y_predict:')
    print( y_predict)
    #print  y_test

    do_metrics(y_test, y_predict)


In [38]:
do_cnn(x,y)

CNN and tf
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


: 

### xgboot

In [33]:
def do_xgboost(x,y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)
    print("xgboost")
    xgb_model = xgb.XGBClassifier().fit(x_train, y_train)
    y_pred = xgb_model.predict(x_test)
    print(classification_report(y_test, y_pred))
    print(metrics.confusion_matrix(y_test, y_pred))
do_xgboost(x,y)

xgboost
              precision    recall  f1-score   support

           0       1.00      0.94      0.97       238
           1       0.94      1.00      0.97       214

    accuracy                           0.97       452
   macro avg       0.97      0.97      0.97       452
weighted avg       0.97      0.97      0.97       452

[[224  14]
 [  1 213]]


### SVM

In [34]:
def do_svm(x,y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)
    clf = svm.SVC()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)

    do_metrics(y_test,y_pred)
do_svm(x,y)

metrics.accuracy_score:
0.9867256637168141
metrics.confusion_matrix:
[[233   5]
 [  1 213]]
metrics.precision_score:
0.9770642201834863
metrics.recall_score:
0.9953271028037384
metrics.f1_score:
0.9861111111111112


### RNN

In [35]:
def do_rnn(x,y):
    global max_document_length
    print("RNN")
    trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.4, random_state=0)
    y_test=testY

    trainX = pad_sequences(trainX, maxlen=max_document_length, value=0.)
    testX = pad_sequences(testX, maxlen=max_document_length, value=0.)
    # Converting labels to binary vectors
    trainY = to_categorical(trainY, nb_classes=2)
    testY = to_categorical(testY, nb_classes=2)

    # Network building
    net = tflearn.input_data([None, max_document_length])
    net = tflearn.embedding(net, input_dim=10240000, output_dim=128)
    net = tflearn.lstm(net, 128, dropout=0.8)
    net = tflearn.fully_connected(net, 2, activation='softmax')
    net = tflearn.regression(net, optimizer='adam', learning_rate=0.001,
                             loss='categorical_crossentropy')

    # Training
    model = tflearn.DNN(net, tensorboard_verbose=0)
    model.fit(trainX, trainY, validation_set=0.1, show_metric=True,
              batch_size=10,run_id="webshell",n_epoch=5)

    y_predict_list=model.predict(testX)
    y_predict=[]
    for i in y_predict_list:
        if i[0] > 0.5:
            y_predict.append(0)
        else:
            y_predict.append(1)

    do_metrics(y_test, y_predict)
do_rnn(x,y)

RNN
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
---------------------------------
Run id: webshell
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 608
Validation samples: 68
--


ResourceExhaustedError: OOM when allocating tensor with shape[10240000,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node Adam/apply_grad_op_0/update_Embedding/W/mul_5 (defined at d:\Anaconda\lib\site-packages\tflearn\helpers\trainer.py:716) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for 'Adam/apply_grad_op_0/update_Embedding/W/mul_5':
  File "d:\Anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\Anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "d:\Anaconda\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "d:\Anaconda\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "d:\Anaconda\lib\site-packages\ipykernel\kernelapp.py", line 583, i

### RF

In [36]:
def do_rf(x,y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=0)
    rf = RandomForestClassifier(n_estimators=50)
    rf.fit(x_train, y_train)
    y_pred = rf.predict(x_test)
    do_metrics(y_test,y_pred)
do_rf(x,y)

metrics.accuracy_score:
0.9734513274336283
metrics.confusion_matrix:
[[228  10]
 [  2 212]]
metrics.precision_score:
0.954954954954955
metrics.recall_score:
0.9906542056074766
metrics.f1_score:
0.9724770642201835


## 预测模型

In [ ]:

def load_files(path):
    files_list=[]
    for r, d, files in os.walk(path):
        for file in files:
            if file.endswith('.php'):
                file_path=path+file
                print("Load %s" % file_path)
                t=load_file(file_path)
                files_list.append(t)
    return  files_list

def check_webshell(clf,dir):
    all=0
    all_php=0
    webshell=0

    webshell_files_list = load_files_re(webshell_dir)
    CV = CountVectorizer(ngram_range=(3, 3), decode_error="ignore", max_features=max_features,
                         token_pattern=r'\b\w+\b', min_df=1, max_df=1.0)
    x = CV.fit_transform(webshell_files_list).toarray()

    transformer = TfidfTransformer(smooth_idf=False)
    transformer.fit_transform(x)


    g = os.walk(dir)
    for path, d, filelist in g:
        for filename in filelist:
            fulepath=os.path.join(path, filename)
            t = load_file(fulepath)
            t_list=[]
            t_list.append(t)
            x2 = CV.transform(t_list).toarray()
            x2 = transformer.transform(x2).toarray()
            y_pred = clf.predict(x2)
            all+=1
            if filename.endswith('.php'):
                all_php+=1
            if y_pred[0] == 1:
                print("%s is webshell" % fulepath)
                webshell+=1

    print("Scan %d files(%d php files),%d files is webshell" %(all,all_php,webshell))

def do_check(x,y,clf):
    clf.fit(x, y)
    print("check_webshell")
    #check_webshell(clf,"../Data/webshell/normal/php/")
    #/Users/maidou/Downloads/webshell-master/php
    check_webshell(clf,check_dir)
